In [4]:
import logging
import time
import random
from datetime import datetime

n_campi = 'CIGR'

data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

logger = logging.getLogger('consolidação')
logger.setLevel(logging.DEBUG)

fHandler = logging.FileHandler(f'logs/{data_e_hora_atuais}_consolidacao.log', mode='w')
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

fHandler.setFormatter(formatter)
logger.addHandler(fHandler)

# logging.basicConfig(level=logging.DEBUG, filename='app.log', filemode='x', format='%(asctime)s - %(levelname)s - %(message)s')
logger.info(f'Iniciando consolidação campus {n_campi}')
# logger.debug('debug message')
# logger.warning('warn message')

In [5]:
import pandas as pd

logger.info(f'carregando palnilha de levantamento fisico campus {n_campi}')
levantamento = pd.read_excel('consolidacao/levantamento/2023_levantamento_CIGR.xlsx')
time.sleep(random.randint(60, 80))

logger.info(f'carregando palnilha de estoque suap campus {n_campi}')
suap = pd.read_excel('consolidacao/suap/2023-08-10_16-07_CIGR_relatorio_almox.xlsx')
time.sleep(random.randint(60, 80))

logger.info(f'tratamdp dados da planilha de levantameto fisico {n_campi}')
time.sleep(random.randint(60, 80))

# removendo itens nulos e deixando apenas os retificados
retificacao = levantamento[levantamento['Retificação'].str.contains('Retificação').fillna(False)]
# lista de itens vencidos
# vencido = levantamento[levantamento['Material Vencido - MV'].str.contains('x').fillna(False)]
# lista de itens não identificados
# nao_identificado = levantamento[levantamento['Sem identificação - SI'].str.contains('x').fillna(False)]

print(levantamento.info())
print(suap.info())

# planilha contendo apenas itens válidos
levantamento_ret = levantamento.drop_duplicates(subset=['Código', 'Retificação'], keep='last')
levantamento_ret = levantamento_ret.rename(columns={'Código': 'codigo'})
# elemento despesa do levantamento
ed_lev = levantamento_ret['codigo'].str[3:5]


# adicionando ed ao data frame geral
# levantamento_ret['ED'] = ed_lev

logger.info(f'removendo dados duplicados')
time.sleep(random.randint(60, 80))
# lista de servidores
servidores = levantamento['Endereço de e-mail'].drop_duplicates(keep='last')
# reindexando uutilizando list comprehension
servidores.index = [x for x in range(0, len(servidores))]

logger.info(f'cruzando dados entre levantamento fisico e suap')
time.sleep(random.randint(60, 80))

levantamento_cross = levantamento_ret.drop(columns=['Material de pouca movimentação - PM', 'Material Vencido - MV',
                                                 'Material próximo ao vencimento - PV', 'Material danificado ou avariado - MD',
                                                 'Material Obsoleto - MO', 'Sem identificação - SI', 'Valor unitário',
                                                 'Campus', 'Retificação', 'Carimbo de data/hora'])
levantamento_cross = levantamento_cross.rename(columns={'Endereço de e-mail': 'servidor', 'Nome': 'descricao_lev',
                                                        'Quantidade': 'qtd_lev'})
levantamento_cross = levantamento_cross[['codigo', 'descricao_lev', 'qtd_lev', 'servidor']]

suap_cross = suap.rename(columns={'descricao': 'descricao_suap', 'qtd': 'qtd_suap'})
suap_cross = suap_cross.drop(columns=['vlr_total', 'vlr_unt'])
display(suap_cross)
display(levantamento_cross)

logger.info(f'consolidando dados')
time.sleep(random.randint(60, 80))

logger.info(f'exportando planilha de consolidação')
time.sleep(random.randint(60, 80))

consolidacao = pd.merge(suap_cross, levantamento_cross, on='codigo', how='outer', indicator=True)
consolidacao.to_excel(f'consolidacao/{n_campi}_consolidacao.xlsx', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Carimbo de data/hora                  179 non-null    datetime64[ns]
 1   Endereço de e-mail                    179 non-null    object        
 2   Retificação                           10 non-null     object        
 3   Código                                179 non-null    object        
 4   Nome                                  179 non-null    object        
 5   Quantidade                            179 non-null    int64         
 6   Campus                                179 non-null    object        
 7   Localização                           66 non-null     object        
 8   Valor unitário                        179 non-null    float64       
 9   Sem identificação - SI                0 non-null      float64       
 10  Ma

,codigo,descricao_suap,und,qtd_suap,ed
0,30.11.922,"Álcool 70. agente desinfetante, Líquido, límpi...",BOMBONA,9,11 - MATERIAL QUIMICO
1,30.14.177,"BOLA DE VÔLEI APLICAÇÃO EM (PRAIA),CIRC. 65 À ...",UN,4,14 - MATERIAL EDUCATIVO E ESPORTIVO
2,30.14.24,BOMBA DE ENCHER BOLA,UN,3,14 - MATERIAL EDUCATIVO E ESPORTIVO
3,30.14.278,COLETE ESPORTIVO POLIESTER.,UN,100,14 - MATERIAL EDUCATIVO E ESPORTIVO
4,30.14.283,"COLETE ESPORTIVO, MATERIAL POLIÉSTER, NUMERADO...",UN,24,14 - MATERIAL EDUCATIVO E ESPORTIVO
...,...,...,...,...,...
159,30.36.532,KIT DE EMERGÊNCIA PARA DESCONTAMINAÇÃO DE AMBI...,Conjunto,1,36 - MATERIAL HOSPITALAR
160,30.42.280,ARCO SERRA 10 E 12 POL. TIPO REGULÁVEL.,UN,1,42 - FERRAMENTAS
161,30.42.877,"JOGO DE CHAVES ALLEN COM 11 PEÇAS, 1,5MM, 2MM,...",JOGO,1,42 - FERRAMENTAS
162,30.44.61,PLACA SETA INDICATIVA DE AGUA PRESSURIZADA (AP).,UN,2,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS


,codigo,descricao_lev,qtd_lev,servidor
0,30.16.2168,"CANETA ESFEROGRÁFICA BIC, COR: PRETA",57,claudeandela.cavalcanti@reitoria.ifpe.edu.br
1,30.16.713,"CANETA ESFEROGRÁFICA, MATERIAL PLÁSTICO, QUANT...",54,claudeandela.cavalcanti@reitoria.ifpe.edu.br
2,30.16.572,PONTA REFIL MARCADOR RECARREGÁVEL PARA QUADRO ...,31,claudeandela.cavalcanti@reitoria.ifpe.edu.br
3,30.16.169,CLIPE DE METAL 2/0,66,claudeandela.cavalcanti@reitoria.ifpe.edu.br
4,30.16.684,"CLIPE II - TAMANHO 8/0, CAIXA COM 25 UN",107,claudeandela.cavalcanti@reitoria.ifpe.edu.br
...,...,...,...,...
174,30.44.61,PLACA SETA INDICATIVA DE AGUA PRESSURIZADA (AP).,4,cmpsg@igarassu.ifpe.edu.br
175,30.17.835,"CARTUCHO DE TONER, P. IMPRESSORA HP PRO 400 CO...",15,cmpsg@igarassu.ifpe.edu.br
176,30.22.143,"SACO PLÁSTICO, 200 L, PCT C/ 100.",86,cmpsg@igarassu.ifpe.edu.br
177,30.16.182,"ESTILETE, CORPO PLÁSTICO, LÂMINA METAL LARGA (...",3,cmpsg@igarassu.ifpe.edu.br


In [6]:
logger.info(f'montando planilha de ajustes ')
time.sleep(random.randint(60, 80))

consolidacao['diferenca'] = consolidacao['qtd_suap'] - consolidacao['qtd_lev']
display(consolidacao)

pedidos = consolidacao[consolidacao['_merge'].str.contains('both')]
pedidos = pedidos[['codigo','ed' ,'descricao_suap', 'qtd_suap', 'qtd_lev', 'diferenca']]

acao = []
for valor in pedidos['diferenca']:
    
    if valor > 0:
        baixa = "pedido no suap"
        acao.append(baixa)
    elif valor < 0:
        baixa = "saida sem pedido"
        acao.append(baixa)
    else:
        baixa = 'adequado'
        acao.append(baixa)

logger.info(f'exportando planilha de ajustes ')
time.sleep(random.randint(60, 80))

pedidos['acao'] = acao
pedidos.to_excel(f'consolidacao/pedidos/{n_campi}_pedidos.xlsx', index=False)
display(pedidos)

,codigo,descricao_suap,und,qtd_suap,ed,descricao_lev,qtd_lev,servidor,_merge,diferenca
0,30.11.922,"Álcool 70. agente desinfetante, Líquido, límpi...",BOMBONA,9,11 - MATERIAL QUIMICO,"Álcool 70. agente desinfetante, Líquido, límpi...",9.0,cmpsg@igarassu.ifpe.edu.br,both,0.0
1,30.11.922,"Álcool 70. agente desinfetante, Líquido, límpi...",BOMBONA,9,11 - MATERIAL QUIMICO,"Álcool 70. agente desinfetante, Líquido, límpi...",9.0,cmpsg@igarassu.ifpe.edu.br,both,0.0
2,30.14.177,"BOLA DE VÔLEI APLICAÇÃO EM (PRAIA),CIRC. 65 À ...",UN,4,14 - MATERIAL EDUCATIVO E ESPORTIVO,"BOLA DE VÔLEI APLICAÇÃO EM (PRAIA),CIRC. 65 À ...",4.0,edivonaldo.araujo@igarassu.ifpe.edu.br,both,0.0
3,30.14.24,BOMBA DE ENCHER BOLA,UN,3,14 - MATERIAL EDUCATIVO E ESPORTIVO,BOMBA DE ENCHER BOLA,4.0,cmpsg@igarassu.ifpe.edu.br,both,-1.0
4,30.14.278,COLETE ESPORTIVO POLIESTER.,UN,100,14 - MATERIAL EDUCATIVO E ESPORTIVO,COLETE ESPORTIVO POLIESTER.,100.0,cmpsg@igarassu.ifpe.edu.br,both,0.0
...,...,...,...,...,...,...,...,...,...,...
168,30.36.532,KIT DE EMERGÊNCIA PARA DESCONTAMINAÇÃO DE AMBI...,Conjunto,1,36 - MATERIAL HOSPITALAR,KIT DE EMERGÊNCIA PARA DESCONTAMINAÇÃO DE AMBI...,1.0,cmpsg@igarassu.ifpe.edu.br,both,0.0
169,30.42.280,ARCO SERRA 10 E 12 POL. TIPO REGULÁVEL.,UN,1,42 - FERRAMENTAS,NaN,NaN,NaN,left_only,NaN
170,30.42.877,"JOGO DE CHAVES ALLEN COM 11 PEÇAS, 1,5MM, 2MM,...",JOGO,1,42 - FERRAMENTAS,"JOGO DE CHAVES ALLEN COM 11 PEÇAS, 1,5MM, 2MM,...",1.0,edivonaldo.araujo@igarassu.ifpe.edu.br,both,0.0
171,30.44.61,PLACA SETA INDICATIVA DE AGUA PRESSURIZADA (AP).,UN,2,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS,PLACA SETA INDICATIVA DE AGUA PRESSURIZADA (AP).,4.0,cmpsg@igarassu.ifpe.edu.br,both,-2.0


,codigo,ed,descricao_suap,qtd_suap,qtd_lev,diferenca,acao
0,30.11.922,11 - MATERIAL QUIMICO,"Álcool 70. agente desinfetante, Líquido, límpi...",9,9.0,0.0,adequado
1,30.11.922,11 - MATERIAL QUIMICO,"Álcool 70. agente desinfetante, Líquido, límpi...",9,9.0,0.0,adequado
2,30.14.177,14 - MATERIAL EDUCATIVO E ESPORTIVO,"BOLA DE VÔLEI APLICAÇÃO EM (PRAIA),CIRC. 65 À ...",4,4.0,0.0,adequado
3,30.14.24,14 - MATERIAL EDUCATIVO E ESPORTIVO,BOMBA DE ENCHER BOLA,3,4.0,-1.0,saida sem pedido
4,30.14.278,14 - MATERIAL EDUCATIVO E ESPORTIVO,COLETE ESPORTIVO POLIESTER.,100,100.0,0.0,adequado
...,...,...,...,...,...,...,...
167,30.36.523,36 - MATERIAL HOSPITALAR,"SABONETEIRA, ALTURA 28,70 CM, LARGURA 12,50 CM...",50,50.0,0.0,adequado
168,30.36.532,36 - MATERIAL HOSPITALAR,KIT DE EMERGÊNCIA PARA DESCONTAMINAÇÃO DE AMBI...,1,1.0,0.0,adequado
170,30.42.877,42 - FERRAMENTAS,"JOGO DE CHAVES ALLEN COM 11 PEÇAS, 1,5MM, 2MM,...",1,1.0,0.0,adequado
171,30.44.61,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS,PLACA SETA INDICATIVA DE AGUA PRESSURIZADA (AP).,2,4.0,-2.0,saida sem pedido
